<a href="https://colab.research.google.com/github/pinballsurgeon/sweeping_analysis/blob/main/emergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## installs

In [2]:
!pip install pytrends

## imports

In [3]:
import pandas 

from pytrends.request import TrendReq
from pytrends         import dailydata


In [4]:
# configurations
config_dict = { 'analysis_start_year'  : 2021
              , 'analysis_start_month' : 1
              , 'analysis_end_year'    : 2021
              , 'analysis_end_month'   : 3
              , 'analysis_country'     : 'US' }


# topics
topics_explicit = ['duck', 'goose']



In [5]:
df_total = pandas.DataFrame()

for topic in topics_explicit:

  df = dailydata.get_daily_data( topic
                               , config_dict['analysis_start_year']
                               , config_dict['analysis_start_month']
                               , config_dict['analysis_end_year']
                               , config_dict['analysis_end_month']
                               , geo = config_dict['analysis_country']
                               , verbose = False )
  
  df_total[topic] = df[topic]

# drop nans
df_total = df_total.dropna()

In [6]:
# recent data set
# df_total.index = pandas.to_datetime(df_total.index)


df_features = df_total[topics_explicit]

In [7]:
df_features

,duck,goose
date,,
2021-01-01,82.81,90.25
2021-01-02,88.36,100.00
2021-01-03,84.64,86.49
2021-01-04,51.84,61.62
2021-01-05,49.00,50.41
...,...,...
2021-03-27,78.21,64.40
2021-03-28,70.50,62.10
2021-03-29,50.40,48.80


In [12]:
# normalize
df_features=(df_features-df_features.mean())/df_features.std()

In [11]:
df_features

,duck,goose
date,,
2021-01-01,1.856863,2.523979
2021-01-02,2.246946,3.261980
2021-01-03,1.985485,2.239376
2021-01-04,-0.319868,0.356906
2021-01-05,-0.519478,-0.491605
...,...,...
2021-03-27,1.533552,0.567331
2021-03-28,0.991653,0.393239
2021-03-29,-0.421078,-0.613470


In [13]:
from operator import index
# normalize it all history (meanly)
df_total = (df_total-df_total.mean())/df_total.std()
df_features_recent = df_total[df_total.index > '1/1/2017']

# normalize it recent history (meanly)
df_features_recent = (df_features_recent-df_features_recent.mean())/df_features_recent.std()

# group by week, average occurences
df_agg = df_features
df_agg_recent = df_features_recent


# print(df_agg.head())

In [14]:
# populate us
df_stk=pandas.DataFrame()
globalst=[]

# navigate hierarchy, each combination of WEEKNUM and FLOWER
for col in df_agg:           # flower
    for val in df_agg.index: # weeknum
        
        
        # time sensitive
        fs_val = df_agg[col][val]                   # full spectrum
        
        # in cases of no recent
        try:
            fs_val_recent = df_agg_recent[col][val] # recent spectrum
        except:
            fs_val_recent = 0
        
        
        # at this combo of flower/weeknum, check which source is greater, as its our limiter
        if fs_val > fs_val_recent:
            fs_max = fs_val
            fs_min = fs_val_recent
            fs_max_src = 'full'
        else:
            fs_max = fs_val_recent
            fs_min = fs_val
            fs_max_src = 'recent'
        
        
        # if max frequency signal greater than 0
        #if fs_max > 0 or fs_min > 0:
        #if fs_max > 0:
        if fs_max > 0.1 and fs_min > 0.1:
            
            # secondary source counter
            sec_i=1
            
            
            # iterate through each instance of max bin occurence, determining number of weeks written
            for x in range(0,int(abs(fs_max) * 100)):
            
            
                # full source
                if fs_max_src == 'full':
                    
                    if sec_i < int(abs(fs_min) * 100):
                    
                        lst=[val, val, col]
                        globalst.append(lst)
                    
                    else:
                        
                        lst=[val, None, col]
                        globalst.append(lst)                    
                    
                # recent source
                if sec_i < int(abs(fs_min) * 100):
                    
                    lst=[val, val, col]
                    globalst.append(lst)
                    
                else:
                        
                    lst=[None, val, col]
                    globalst.append(lst)                     
                    
                    
                sec_i+=1
                #print(sec_i)
                #print(df_agg[col][val])
                #print(df_agg_recent[col][val])
                
                
                #lst=[val, col]
                globalst.append(lst)
       
    
        #df_stk=df_stk.append(lst)

df_flat = pandas.DataFrame(globalst
                      ,columns=['WeekNum'
                               ,'WeekNum_Rct'
                               ,'Flower_Tot'])

In [8]:
df_flat.value_counts()

NameError: ignored

In [ ]:
#df_flat
df_stats

,Flower_Tot,WeekNum mean,WeekNum count,WeekNum_Rct mean,WeekNum_Rct count
1,goose,2021-02-07 19:37:13.696408832,236957,2021-02-07 22:39:30.710275584,462961
0,duck,2021-02-12 07:12:03.656068608,243409,2021-02-12 09:11:38.585199360,475544


In [ ]:
df_stats = df_flat.groupby('Flower_Tot').agg(['mean', 'count']).reset_index()
df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]
print(df_stats.columns)
df_stats = df_stats.sort_values(by='WeekNum mean',ascending=True)

#df_stats['Name'] = '00' + str((df_stats['WeekNum mean']).astype(int)) #+ df_stats['Flower']
#df_stats['Name'] = ( ( df_stats['WeekNum mean'] * 100) + 1000).astype('int').astype('str') + ' '+ df_stats['Flower_Tot']

i = 1
for index, row in df_stats.iterrows():
    #print(row['Flower'], row['Name'])
    df_flat['Flower_Tot'] = df_flat['Flower_Tot'].replace(row['Flower_Tot'], row['Name'])
    #df_flat['Flower'] = df_flat['Flower'].replace(row['Flower'], str(i) + row['Flower'])
    i+=1

print(df_flat)

Index(['Flower_Tot', 'WeekNum mean', 'WeekNum count', 'WeekNum_Rct mean',
       'WeekNum_Rct count'],
      dtype='object')


KeyError: ignored